In [1]:
# to access root modules
if __name__ == "__main__" and __package__ is None:
    from sys import path
    from os.path import dirname as dir
    path.append(dir(path[0])+'/dash_app')
    __package__ = "dash_app"

In [44]:
import pandas as pd
import numpy as np
import datetime
import dateutil
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from charts import stations_map, map_columns, date_to_day_number, percentile_series
from settings import *


In [3]:
data_path = DATA_PATH+'Terrestrial_Domain/4.4SoilMoisture/Figure4.4/'
data_csv = data_path + 'Figure4.4_data.csv'

In [4]:
"""
Tidy data for charts
"""
xls = pd.ExcelFile(
    data_path+'Figure4.4_DailyAverageSM.xlsx')
original_df = pd.read_excel(xls, 'DailyAverage')
original_df

,stno,month,day,smd,years,Unnamed: 5,stno.1,month.1,day.1,smd.1,years.1
0,305,1,1,-3.0850,38,NaN,532,1,1,-0.9400,38
1,305,1,2,-3.5950,38,NaN,532,1,2,-0.7325,38
2,305,1,3,-3.8850,38,NaN,532,1,3,-1.4400,38
3,305,1,4,-4.2025,38,NaN,532,1,4,-0.8575,38
4,305,1,5,-3.5325,38,NaN,532,1,5,-0.4825,38
...,...,...,...,...,...,...,...,...,...,...,...
360,305,12,27,-2.5025,38,NaN,532,12,27,-1.4950,38
361,305,12,28,-2.8875,38,NaN,532,12,28,-0.5900,38
362,305,12,29,-3.0700,38,NaN,532,12,29,-1.5350,38
363,305,12,30,-3.5000,38,NaN,532,12,30,-1.4925,38


In [22]:
columns_dict_valentia = {
#                'datetime':'date',
               'average__daily__soil_moisture_deficit':"smd",
               'month':"month",
               'day':"day",
                  }
columns_dict_dublin = {
#                'datetime':'date.1',
               'average__daily__soil_moisture_deficit':"smd.1",
               'month':"month.1",
               'day':"day.1",
                  }
valentia_df = map_columns(columns_dict_valentia, original_df)
valentia_df['start_year']='1980'
valentia_df['end_year']='2019'
valentia_df['location'] = 'Valentia'
dublin_df = map_columns(columns_dict_dublin, original_df)
dublin_df['start_year']='1980'
dublin_df['end_year']='2019'
dublin_df['location'] = 'Dublin_Airport'
df = pd.concat([valentia_df,dublin_df])
df.reset_index(drop=True, inplace=True)
# df['day_of_year'] = df['datetime'].dt.dayofyear
# df.to_csv(data_csv)

In [23]:
data_path = DATA_PATH+'Terrestrial_Domain/4.4SoilMoisture/Figure4.4/'
data_csv = data_path + 'Figure4.4_data.csv'
df = pd.read_csv(data_csv, index_col=0)
df

,average__daily__soil_moisture_deficit,month,day,start_year,end_year,location
0,-3.0850,1,1,1980,2019,Valentia
1,-3.5950,1,2,1980,2019,Valentia
2,-3.8850,1,3,1980,2019,Valentia
3,-4.2025,1,4,1980,2019,Valentia
4,-3.5325,1,5,1980,2019,Valentia
...,...,...,...,...,...,...
725,-1.4950,12,27,1980,2019,Dublin_Airport
726,-0.5900,12,28,1980,2019,Dublin_Airport
727,-1.5350,12,29,1980,2019,Dublin_Airport
728,-1.4925,12,30,1980,2019,Dublin_Airport


In [24]:
df['year']=df['end_year']
df['datetime']=pd.to_datetime(df[['year','month','day']])
valentia_df = df.loc[(df['location'] == 'Valentia')]
dublin_df = df.loc[(df['location'] == 'Dublin_Airport')]

In [25]:
valentia_trace = go.Scatter(x=valentia_df['datetime'],
                            y=valentia_df['average__daily__soil_moisture_deficit'],
                         name='Valentia',
                         mode='markers+lines',
                         marker=dict(color=TIMESERIES_COLOR_1,
                                     size=5,
                                     opacity=0.5),
                         line=dict(color=TIMESERIES_COLOR_1,
                                      width=1),
                         hovertemplate='%{x|%d %b}<br>' +
                         '<b>Valentia</b><br>' +
                         'SMD: %{y:.2f} mm<br>' +
                         '<extra></extra>'
                         )
dublin_trace = go.Scatter(x=dublin_df['datetime'],
                            y=dublin_df['average__daily__soil_moisture_deficit'],
                         name='Dublin Airport',
                         mode='markers+lines',
                         marker=dict(color=TIMESERIES_COLOR_2,
                                     size=5,
                                     opacity=0.5),
                         line=dict(color=TIMESERIES_COLOR_2,
                                      width=1),
                         hovertemplate='%{x|%d %b}<br>' +
                         '<b>Dublin Airport</b><br>' +
                         'SMD: %{y:.2f} mm<br>' +
                         '<extra></extra>'
                         )

figure_4_4 = go.Figure(data=[valentia_trace, dublin_trace], layout=TIMESERIES_LAYOUT)
figure_4_4.update_layout(
    yaxis=dict(
        title='Soil Moisture Deficet, SMD (mm)',),
    xaxis=dict(
        title="Month",
        dtick="M1",
        tickformat="%b",
        ticklabelmode="period",
              ))
figure_4_4

In [29]:
"""
Figure 4.5
"""
data_path = DATA_PATH+'Terrestrial_Domain/4.4SoilMoisture/Figure4.5/'
data_csv = data_path + 'Figure4.5_data.csv'

In [41]:
"""
Tidy data for charts
"""
xls = pd.ExcelFile(
    data_path+'Figure4.5_HeatMapSM.xlsx')
original_df = pd.read_excel(xls, 'RawData')
columns_dict_valentia = {
               'datetime':'date',
               'mean__daily__soil_moisture_deficit':"smd_md",
                  }
columns_dict_dublin = {
               'datetime':'date.1',
               'mean__daily__soil_moisture_deficit':"smd_md.1",
                  }
df = map_columns(columns_dict_valentia, original_df)
series =pd.Series(df['mean__daily__soil_moisture_deficit'].values,index=df['datetime'])
# Calc mean monthly 
mean_monthly = series.resample('M').mean()
valentia_df = pd.DataFrame()

valentia_df['mean__monthly__soil_moisture_deficit'] = mean_monthly
valentia_df['datetime'] = valentia_df.index
valentia_df['location'] = 'Valentia'
valentia_df
df = map_columns(columns_dict_dublin, original_df)
series =pd.Series(df['mean__daily__soil_moisture_deficit'].values,index=df['datetime'])
# Calc mean monthly 
mean_monthly = series.resample('M').mean()
dublin_df = pd.DataFrame()
dublin_df['mean__monthly__soil_moisture_deficit'] = mean_monthly
dublin_df['datetime'] = dublin_df.index
dublin_df['location'] = 'Dublin_Airport'

df = pd.concat([valentia_df,dublin_df])
df = df[['datetime','mean__monthly__soil_moisture_deficit','location']]
df.reset_index(drop=True, inplace=True)
# df.to_csv(data_csv)

In [45]:
data_path = DATA_PATH+'Terrestrial_Domain/4.4SoilMoisture/Figure4.5/'
data_csv = data_path + 'Figure4.5_data.csv'
df = pd.read_csv(data_csv, index_col=0)
df = df.loc[(df['location'] == 'Dublin_Airport')]
df['datetime'] = pd.to_datetime(df['datetime'])
df['percentile'] = percentile_series(df,'mean__monthly__soil_moisture_deficit')
df

,datetime,mean__monthly__soil_moisture_deficit,location,percentile
480,1980-01-31,-1.025806,Dublin_Airport,5
481,1980-02-29,-0.820690,Dublin_Airport,5
482,1980-03-31,1.887097,Dublin_Airport,25
483,1980-04-30,20.710000,Dublin_Airport,50
484,1980-05-31,50.812903,Dublin_Airport,95
...,...,...,...,...
955,2019-08-31,16.870968,Dublin_Airport,50
956,2019-09-30,17.210000,Dublin_Airport,50
957,2019-10-31,-0.593548,Dublin_Airport,5
958,2019-11-30,-3.496667,Dublin_Airport,0


In [ ]:
months=[1,2,3,4,5,6,7,8,9,10,11,12]
months=['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
df_months

In [56]:
trace = go.Heatmap(
    z=df['percentile'],
    x=df['datetime'].dt.month,
    y=df['datetime'].dt.year,
    text=df['mean__monthly__soil_moisture_deficit'],
    colorscale=PERCENTILE_COLORSCALE, 
    colorbar=dict(
                tickmode='array',
                thickness=10,
                len=0.9,
        
    title='<b>Percentile</b> (mm)',
            ticktext=[
                '<b>Min.</b> (-3.6)', 
                '<b> 5%</b> (-1.9)', 
                '<b>25%</b> (0.4)', 
                '<b>50%</b> (6.9)',
                '<b>75%</b> (23.3)', 
                '<b>95%</b> (49.8)'],
            tickvals=[0,5,24,47, 71, 90]  
    ),
    hovertemplate='%{x} %{y}<br>'+
    'Mean SMD: %{text:.2f} mm<extra></extra>'
)
figure_4_5 = go.Figure(data=trace, layout=TIMESERIES_LAYOUT)
figure_4_5.update_layout(
yaxis=dict(
    title='Year',
    range=[1980, 2020]
),
xaxis=dict(
    title='Month',
    ticktext=['Jan','Feb','Mar','Apr','May','June','July','Aug','Sep','Oct','Nov','Dec'],
    showgrid=False,
   tickvals=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
))

# Include white lines as xaxis gridlines
x0=1.5
for i in range(0,11):
    figure_4_5.add_shape(type='line',
                        x0=x0, y0=1979.5, x1=x0, y1=2019.5,
                        line=dict(color='White', width=3))
    x0+=1

figure_4_5

In [ ]:
"""
Figure 4.4
"""

"""
Below is how to generate data from raw. Not used
"""

In [ ]:
"""
Tidy data for charts
"""
xls = pd.ExcelFile(
    data_path+'Figure4.4_DailyAverageSM.xlsx')
original_df = pd.read_excel(xls, 'RawData')
columns_dict_valentia = {
               'datetime':'date',
               'mean__daily__soil_moisture_deficit':"smd_md",
                  }
columns_dict_dublin = {
               'datetime':'date.1',
               'mean__daily__soil_moisture_deficit':"smd_md.1",
                  }
valentia_df = map_columns(columns_dict_valentia, original_df)
valentia_df['location'] = 'Valentia'
dublin_df = map_columns(columns_dict_dublin, original_df)
dublin_df['location'] = 'Dublin_Airport'
df = pd.concat([valentia_df,dublin_df])
# df['day_of_year'] = df.apply (lambda row: date_to_day_number(row,standard30=False), axis=1)
df.reset_index(drop=True, inplace=True)
df['day_of_year'] = df['datetime'].dt.dayofyear
# df.to_csv(data_csv)

In [ ]:
df

In [ ]:
data_path = DATA_PATH+'Terrestrial_Domain/4.4SoilMoisture/Figure4.4/'
data_csv = data_path + 'Figure4.4_data.csv'
df = pd.read_csv(data_csv, index_col=0)
df

In [ ]:
valentia_df = df.loc[(df['location'] == 'Valentia')]
valentia_df_days = valentia_df.groupby(valentia_df['day_of_year']).mean()
valentia_df_days['date'] = datetime.datetime(1900, 1, 1) + datetime.timedelta(valentia_df['day_of_year'] - 1)
dublin_df = df.loc[(df['location'] == 'Dublin_Airport')]
dublin_df_days = dublin_df.groupby(dublin_df['day_of_year']).mean()
valentia_df_days

In [ ]:
valentia_trace = go.Scatter(x=valentia_df_days.index,
                            y=valentia_df_days['mean__daily__soil_moisture_deficit'],
                         name='Valentia',
                         mode='markers+lines',
                         marker=dict(color=TIMESERIES_COLOR_1,
                                     size=5,
                                     opacity=0.5),
                         line=dict(color=TIMESERIES_COLOR_1,
                                      width=1),
                         hovertemplate='%{text}<br>' +
                         '<b>Valentia</b><br>' +
                         'SMD: %{y:.2f} mm<br>' +
                         '<extra></extra>'
                         )
dublin_trace = go.Scatter(x=dublin_df_days.index,
                            y=dublin_df_days['mean__daily__soil_moisture_deficit'],
                         name='Dublin Airport',
                         mode='markers+lines',
                         marker=dict(color=TIMESERIES_COLOR_2,
                                     size=5,
                                     opacity=0.5),
                         line=dict(color=TIMESERIES_COLOR_2,
                                      width=1),
                         hovertemplate='%{x}<br>' +
                         '<b>Dublin Airport</b><br>' +
                         'SMD: %{y:.2f} mm<br>' +
                         '<extra></extra>'
                         )

figure_4_4 = go.Figure(data=[valentia_trace, dublin_trace], layout=TIMESERIES_LAYOUT)
figure_4_4.update_layout(
    yaxis=dict(title='Soil Moisture Deficet, SMD (mm)'),
    xaxis=dict(title="Month"))
figure_4_4